- 참고 : https://sieon-dev.tistory.com/15

## 1. 모듈 import (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)
- pandas
- numpy
- matplotlib
- re
- konlpy
- tensorflow
- seaborn
- pandas_profiling

In [14]:
# pip install konlpy

     |████████████████████████████████| 19.4 MB 2.6 MB/s 
     |████████████████████████████████| 448 kB 43.0 MB/s 


In [1]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 35.13 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 519 kB/s 
     |████████████████████████████████| 448 kB 69.9 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-03-27 05:50:35--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loc

In [3]:
pip install exchange-calendars

     |████████████████████████████████| 150 kB 11.0 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.6.1-py3-none-any.whl size=180505 sha256=5d65e27b425323b5e57c6c7976685e21fa00933955780107ec4dce66d27291a4
  Stored in directory: /root/.cache/pip/wheels/db/70/a2/5e1d8d0873feb8cb9808f3b55e8f270698b742db54a4b8ad2e
Successfully built exchange-calendars


In [7]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from konlpy.tag import Mecab
import MeCab
from tqdm import tqdm
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pandas_profiling
import sklearn
from sklearn.model_selection import train_test_split
import collections
from collections import Counter
import exchange_calendars as ecals
import datetime
import time
from wordcloud import WordCloud
import matplotlib as mpl
import nltk

### 1-1. 파이썬 & 모듈 버전 확인 (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)

In [8]:
print(" python version : ", sys.version, "\n", "-" * 100)
print(" pandas version : ", pd.__version__, "\n", "-" * 100)
print(" numpy version : ", np.__version__, "\n", "-" * 100)
print(" re version : ", re.__version__, "\n", "-" * 100)
print(" tensorflow version : ", tensorflow.__version__, "\n", "-" * 100)
print(" seaborn version : ", sns.__version__, "\n", "-" * 100)
print(" sklearn version : ", sklearn.__version__, "\n", "-" * 100)
print(" exhange_calendars version : ", ecals.__version__, "\n", "-" * 100)
print(" nltk version : ", nltk.__version__, "\n", "-" * 100)

 python version :  3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0] 
 ----------------------------------------------------------------------------------------------------
 pandas version :  1.3.5 
 ----------------------------------------------------------------------------------------------------
 numpy version :  1.21.5 
 ----------------------------------------------------------------------------------------------------
 re version :  2.2.1 
 ----------------------------------------------------------------------------------------------------
 tensorflow version :  2.8.0 
 ----------------------------------------------------------------------------------------------------
 seaborn version :  0.11.2 
 ----------------------------------------------------------------------------------------------------
 sklearn version :  1.0.2 
 ----------------------------------------------------------------------------------------------------
 exhange_calendars version :  3.6.1 
 ----------------

### 1-2. 컴퓨터 사양 확인

![](2022-03-24-13-31-45.png)

## 2. 데이터 불러오기

In [9]:
from google.colab import files
up = files.upload()

Saving 덕산네오룩스(213420).xlsx to 덕산네오룩스(213420).xlsx


In [10]:
up = files.upload()

Saving Newsfile3.csv to Newsfile3.csv


In [11]:
# 뉴스데이터 불러오기
newsdata = pd.read_csv('Newsfile3.csv', encoding='euc-kr')
newsdata.tail(3)

,date,title
58439,20151019,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,20151019,[반도체] 2016 반도체 투자 감소 예상 - 미래
58441,20151019,"제주반도체, '해외투자유치설' 조회공시 요구"


### 2-1. 휴장일 데이터 매칭

In [12]:
# newsdata의 datatype 확인
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
dtypes: int64(1), object(1)
memory usage: 913.3+ KB


- type 변환

In [13]:
# datatype : int -> string
newsdata = newsdata.astype({'date':'string'})
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  string
 1   title   58442 non-null  object
dtypes: object(1), string(1)
memory usage: 913.3+ KB


In [14]:
# datatype : string -> datetime
newsdata['date'] = pd.to_datetime(newsdata['date'])
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    58442 non-null  datetime64[ns]
 1   title   58442 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 913.3+ KB


- 뉴스데이터의 처음 날짜와 마지막 날짜 확인

In [15]:
first_date = newsdata['date'][0]
last_date = newsdata['date'][-1:]
print(first_date, last_date)

2022-03-16 00:00:00 58441   2015-10-19
Name: date, dtype: datetime64[ns]


In [16]:
# newsdata -> newsdata_all : 처음 날짜와 마지막 날짜가 포함된 데이터로 이름 변결
newsdata_all = newsdata[newsdata['date'].between('2015-10-19', '2022-03-16')]
newsdata_all

,date,title
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”"
3,2022-03-16,"[특징주] 코리아써키트, 반도체 패키지기판 성장 기대감에 5%↑"
4,2022-03-16,"[특징주] 반도체 대장주 강세…SK하이닉스, 2% 넘게 상승"
...,...,...
58437,2015-10-19,세계 반도체 M&A 1006억달러
58438,2015-10-19,반도체社 인수·합병 봇물…올 들어 1000억달러 돌파
58439,2015-10-19,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,2015-10-19,[반도체] 2016 반도체 투자 감소 예상 - 미래


- 리스트로 변환

In [17]:
# 'date' 컬럼 리스트로 변경
news = newsdata_all['date'].to_list()
news[:5]

[Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00')]

### 2-2. 한국 거래소 개장일 확인

In [18]:
# 한국코드
XKRX = ecals.get_calendar("XKRX")       
# 2021-01-01은 개장일인지 확인 : 'False' -> 휴장일 , 'True' -> 개장일
print("2021-01-01 개장 확인: ", XKRX.is_session('2021-01-01'))    
# 오늘(22-03-24)은 개장일인지 확인 :'False' -> 휴장일 , 'True' -> 개장일
print("오늘날짜 개장일 확인: ", XKRX.is_session(datetime.date.today().strftime('%Y-%m-%d')))  
# 다음 개장일은 언제인지 확인
print("다음 개장일 날짜 확인: ", XKRX.next_open(pd.Timestamp.today().strftime('%Y-%m-%d')))

2021-01-01 개장 확인:  False
오늘날짜 개장일 확인:  False
다음 개장일 날짜 확인:  2022-03-28 00:00:00+00:00


In [19]:
# 뉴스데이터 날짜 다음날에 해당하는 개장일 날짜 확인
next_open_list = []

for day in news:
    next_open_date = XKRX.next_open(day)
    next_open_list.append(next_open_date)

In [20]:
next_open_list[:5]

[Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC')]

In [21]:
# 뉴스나온 날짜 다음 개장일 컬럼 추가
newsdata_all['NextOpenDate'] = next_open_list
newsdata_all.head(3)

,date,title,NextOpenDate
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17 00:00:00+00:00
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17 00:00:00+00:00
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17 00:00:00+00:00


In [22]:
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   date          58442 non-null  datetime64[ns]     
 1   title         58442 non-null  object             
 2   NextOpenDate  58442 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), object(1)
memory usage: 1.8+ MB


- type 변환

In [23]:
newsdata_all = newsdata_all.astype({'NextOpenDate':'string'})
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   NextOpenDate  58442 non-null  string        
dtypes: datetime64[ns](1), object(1), string(1)
memory usage: 1.8+ MB


In [24]:
a = newsdata_all['NextOpenDate'][0]
b = a.split(' ')[0]
b

'2022-03-17'

In [25]:
newsdata_all['NextOpenDate'] = newsdata_all['NextOpenDate'].apply(lambda x: x.split(' ')[0])
newsdata_all.head(3)

,date,title,NextOpenDate
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17


In [26]:
newsdata_all['NextOpenDate'] = pd.to_datetime(newsdata_all['NextOpenDate'])
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   NextOpenDate  58442 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 3.8+ MB


In [27]:
# 개장일 기준 'Year' 컬럼 추가
newsdata_all['NextOpenDateYear'] = newsdata_all['NextOpenDate'].dt.year
newsdata_all.head(3)

,date,title,NextOpenDate,NextOpenDateYear
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17,2022
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17,2022
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17,2022


In [28]:
# col의 인덱스 변경
col = newsdata_all.columns.to_numpy()
col

array(['date', 'title', 'NextOpenDate', 'NextOpenDateYear'], dtype=object)

In [29]:
col = col[[0,3,2,1]]
col

array(['date', 'NextOpenDateYear', 'NextOpenDate', 'title'], dtype=object)

In [30]:
newsdata_all = newsdata_all[col]
newsdata_all.head(3)

,date,NextOpenDateYear,NextOpenDate,title
0,2022-03-16,2022,2022-03-17,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,2022,2022-03-17,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”
2,2022-03-16,2022,2022-03-17,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”"


## 3. 데이터 확인

In [31]:
# 데이터 갯수 : 58442개
# 컬럼 갯수 : 4개
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              58442 non-null  datetime64[ns]
 1   NextOpenDateYear  58442 non-null  int64         
 2   NextOpenDate      58442 non-null  datetime64[ns]
 3   title             58442 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 4.2+ MB


- 결측치 확인

In [32]:
# 결측치 확인
newsdata_all.isnull().sum()

date                0
NextOpenDateYear    0
NextOpenDate        0
title               0
dtype: int64

## 4. 기업별 '현재가', '당일종가-어제종가', '상승률', 'label' 데이터와 뉴스데이터 병합

### 4-1. 반도체 기업 데이터 불러오기 : 005290

In [33]:
df1 = pd.read_excel('덕산네오룩스(213420).xlsx')
df1.head(2)

,Unnamed: 0,Unnamed: 0.1,일자,현재가,당일종가-어제종가,상승률,label
0,0,0,20220314,39050,-1550,-0.038177,0
1,1,1,20220311,40600,-550,-0.013366,0


In [34]:
df1.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
df1.head(2)

,일자,현재가,당일종가-어제종가,상승률,label
0,20220314,39050,-1550,-0.038177,0
1,20220311,40600,-550,-0.013366,0


- 긍정 & 부정 라벨링
    - 상승률이 0보다 크면 `1`로 매칭
    - 상승률이 0보다 작으면 `0`으로 매칭

In [35]:
df1['label'] = df1.apply(lambda x: 1 if x['상승률'] >= 0 else 0, axis=1)
df1

,일자,현재가,당일종가-어제종가,상승률,label
0,20220314,39050,-1550,-0.038177,0
1,20220311,40600,-550,-0.013366,0
2,20220310,41150,1150,0.028750,1
3,20220308,40000,100,0.002506,1
4,20220307,39900,-1300,-0.031553,0
...,...,...,...,...,...
1739,20150212,11774,-526,-0.042764,0
1740,20150211,12300,176,0.014517,1
1741,20150210,12124,-852,-0.065660,0
1742,20150209,12976,150,0.011695,1


- 라벨링 분포 확인

In [36]:
plt.savefig('test.png')
sns.countplot(x='label', data=df1)

- data type 변경

In [37]:
# data type : int -> string
df1 = df1.astype({'일자':'string'})
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   일자         1744 non-null   string 
 1   현재가        1744 non-null   int64  
 2   당일종가-어제종가  1744 non-null   int64  
 3   상승률        1744 non-null   float64
 4   label      1744 non-null   int64  
dtypes: float64(1), int64(3), string(1)
memory usage: 68.2 KB


In [38]:
# data type : string -> datetime
df1['일자'] = pd.to_datetime(df1['일자'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   일자         1744 non-null   datetime64[ns]
 1   현재가        1744 non-null   int64         
 2   당일종가-어제종가  1744 non-null   int64         
 3   상승률        1744 non-null   float64       
 4   label      1744 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 68.2 KB


### 4-2. 뉴스데이터 컬럼명 변경

In [39]:
# 덕산네오룩스의 데이터프레임과 병합하기 위해 컬럼명 변경
newsdata_all.rename(columns={'NextOpenDate':'일자'}, inplace=True)
newsdata_all.head(2)

,date,NextOpenDateYear,일자,title
0,2022-03-16,2022,2022-03-17,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,2022,2022-03-17,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”


### 4-3. 뉴스데이터와 덕산네오룩스 주가데이터 병합

In [40]:
merge_data = pd.merge(newsdata_all, df1, on='일자', how='inner')
merge_data.head(2)

,date,NextOpenDateYear,일자,title,현재가,당일종가-어제종가,상승률,label
0,2022-03-13,2022,2022-03-14,전함 포템킨·오데사·네온사인…반도체는 왜 또 위기일까[인더독],39050,-1550,-0.038177,0
1,2022-03-13,2022,2022-03-14,"한국기업, 반도체·전자 부품 조달 ‘빨간불’",39050,-1550,-0.038177,0


- '일자', 'title', 'label'만 남긴 나머지 컬럼 삭제

In [41]:
merge_data = merge_data[['일자', 'title', 'label']]
merge_data.head(5)

,일자,title,label
0,2022-03-14,전함 포템킨·오데사·네온사인…반도체는 왜 또 위기일까[인더독],0
1,2022-03-14,"한국기업, 반도체·전자 부품 조달 ‘빨간불’",0
2,2022-03-14,“삼성·SK만 잘하면 뭐해?” 한국 반도체 진짜 ‘약점’이 바로 [비즈360],0
3,2022-03-14,"""반도체 일본꼴 날 수도…수도권 대학 반도체학과 정원 풀어야"" [당선인에 바란다]",0
4,2022-03-14,"[우크라 침공] 러시아, 반도체소자 등 500개 품목 대외수출 금지·제한",0


## 5. 데이터 전처리
    - 한글 이외의 문자열은 빈칸(`''`)처리
    - 품사 중 명사만 추출하여 'nouns' 컬럼에 삽입

In [42]:
okt = Okt()
n_ = []
title_rename = []

for i in range(len(merge_data)):
    if (i % 10000 == 0):
        print(i, "단계 완료")
    title_rename.append(re.sub("[\(\[].*?[\)\]]", "", merge_data.iloc[i]['title']))
    n_.append(' '.join(okt.nouns(merge_data.iloc[i]['title'])))
merge_data['nouns'] = n_
merge_data['title'] = title_rename
merge_data = merge_data[merge_data['nouns'] != '']

# 2차 불용어 제거
merge_data['title'] = merge_data['title'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]','')
merge_data['title'].replace('', np.nan, inplace=True)
merge_data = merge_data.dropna(how='any')

0 단계 완료
10000 단계 완료
20000 단계 완료
30000 단계 완료
40000 단계 완료
50000 단계 완료


In [43]:
merge_data.head(3)

,일자,title,label,nouns
0,2022-03-14,전함포템킨오데사네온사인반도체는왜또위기일까,0,전함 포템킨 오데사 네온사인 반도체 왜 또 위기 인 더독
1,2022-03-14,한국기업반도체전자부품조달빨간불,0,한국 기업 반도체 전자 부품 조달 불
2,2022-03-14,삼성만잘하면뭐해한국반도체진짜약점이바로,0,삼성 만 뭐 한국 반도체 진짜 약점 이 바로 비즈


In [44]:
news_labeling = merge_data.copy()

In [45]:
# 결측치 확인
news_labeling.dropna(how='any', inplace=True)
news_labeling.shape

(58343, 4)

- label 분포 확인

In [46]:
sns.countplot(x='label', data=news_labeling)

In [47]:
# 라벨링 값 개수 확인
news_labeling['label'].value_counts()

1    29953
0    28390
Name: label, dtype: int64

## 6. train_data, test_data 분할

In [48]:
# train, test set 분리
train_data, test_data = train_test_split(news_labeling)

### 6-1. train_data

In [49]:
# train_data 확인
train_data.head(2)

,일자,title,label,nouns
42839,2018-06-01,올해설비투자조원전망전년반도체디스플레이편,1,올해 설비 투자 전망 년 반도체 디스플레이 편
21554,2020-07-30,반도체훈풍에로체시스템즈시설투자빛볼까,1,반도체 훈풍 로체 시스 템즈 시설 투자 빛


In [50]:
# train_data shape 확인
train_data.shape

(43757, 4)

### 6-2. test_data

In [51]:
# test_data 확인
test_data.head(2)

,일자,title,label,nouns
33993,2019-04-25,이재용비메모리육성공언반도체비전으로구체화,0,이재용 비메 모리 육성 공언 반도체 구체화
21333,2020-08-04,삼성전자반도체사업장코로나검사소자체운영,0,삼성 전자 반도체 사업 코로나 검사 소 자체 운영


In [52]:
# test_data shape 확인
test_data.shape

(14586, 4)

## 7. 감성사전 구축

- 한 글자인 단어는 제외. 두 글자 이상인 단어들의 점수 초기화 진행

In [53]:
vocab = {}
count = 0
for i in news_labeling['nouns']:
    i = i.split(' ')
    # news_labeling에 있는 데이터의 양만큼 순회
    for j in range(len(i)):
        # i 번째 행의 리스트에 있는 j번째 단어의 길이가 1 이하일 경우 count에 1 추가
        if i[j] in vocab or len(i[j]) <= 1:
            count += 1
            pass
        else:
            # i 번째 행의 리스트에 있는 j번째 단어의 길이가 1 이상일 경우 0
            vocab[i[j]] = 0

In [54]:
vocab

{'전함': 0,
 '포템킨': 0,
 '오데사': 0,
 '네온사인': 0,
 '반도체': 0,
 '위기': 0,
 '더독': 0,
 '한국': 0,
 '기업': 0,
 '전자': 0,
 '부품': 0,
 '조달': 0,
 '삼성': 0,
 '진짜': 0,
 '약점': 0,
 '바로': 0,
 '비즈': 0,
 '일본': 0,
 '수도': 0,
 '수도권': 0,
 '대학': 0,
 '학과': 0,
 '정원': 0,
 '당선인': 0,
 '침공': 0,
 '러시아': 0,
 '반도체소자': 0,
 '품목': 0,
 '대외': 0,
 '수출': 0,
 '금지': 0,
 '제한': 0,
 '호국': 0,
 '통제': 0,
 '공급': 0,
 '우려': 0,
 '애플': 0,
 '수혜': 0,
 '가능성': 0,
 '전기차': 0,
 '원가': 0,
 '상승': 0,
 '비상': 0,
 '배터리': 0,
 '철강': 0,
 '고공': 0,
 '행진': 0,
 '경제': 0,
 '제재': 0,
 '보복': 0,
 '시작': 0,
 '소자': 0,
 '속보': 0,
 '발표': 0,
 '포함': 0,
 '전쟁': 0,
 '가스': 0,
 '생산': 0,
 '중단': 0,
 '네온': 0,
 '절반': 0,
 '대란': 0,
 '심화': 0,
 '파문': 0,
 '대전': 0,
 '조짐': 0,
 '격화': 0,
 '인텔': 0,
 '계속': 0,
 '윤석열': 0,
 '공약': 0,
 '실현': 0,
 '집적회로': 0,
 '상보': 0,
 '산다': 0,
 '두산': 0,
 '통큰': 0,
 '강행': 0,
 '이유': 0,
 '브로드컴': 0,
 '주주': 0,
 '환원': 0,
 '기대': 0,
 '조치': 0,
 '여파': 0,
 '변신': 0,
 '승부': 0,
 '과연': 0,
 '자등': 0,
 '인력': 0,
 '양성': 0,
 '기금': 0,
 '조성': 0,
 '도로': 0,
 '지수': 0,
 '하락': 0,
 '악재': 0,
 '코스피': 0,


### 7-1. train_data

- 상승비율과 하락비율을 정의해준 다음 라벨 값이 1이면 하락 비율을 각 단어에 더해주고 라벨값이 0이면 상승 비율을 차감
- 라벨값이 1이면 상승인데 왜 상승 비율을 더해주지 않았을지 : 라벨값이 1인 데이터가 아닌 데이터보다 훨씬 많다면 해당 단어들의 점수가 너무 커져서 점수가 고르지 못한 감성사전이 만들어지기 때문에 정규화했기 때문

In [55]:
up = 29202
down = 29141

up_ratio = up/(up+down)
down_ratio = down/(up+down)

for i,w in enumerate(train_data['nouns']):
    w = w.split(' ')
    if (train_data.iloc[i]['label']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] - up_ratio

In [56]:
# 감성 점수 확인
vocab

{'전함': 0,
 '포템킨': 0,
 '오데사': 0,
 '네온사인': 0,
 '반도체': 600.9403013214303,
 '위기': -6.230019025418763,
 '더독': 0.9989544589753698,
 '한국': 27.433316764650115,
 '기업': 31.178204754640234,
 '전자': 36.041873061046054,
 '부품': 20.69888418490635,
 '조달': 12.468110998748763,
 '삼성': -67.51389198361694,
 '진짜': -5.515160344857137,
 '약점': -1.5026138525615753,
 '바로': -0.00418216409852068,
 '비즈': -4.035548394837428,
 '일본': 25.26423049894578,
 '수도': 5.957132817990155,
 '수도권': -4.517251426906396,
 '대학': 5.979089179507394,
 '학과': -6.037639476886696,
 '정원': -0.5026138525615754,
 '당선인': -1.0010455410246302,
 '침공': 2.9926812128275895,
 '러시아': 2.4775208679704535,
 '반도체소자': -6.006273246147782,
 '품목': 6.398059750098548,
 '대외': -8.0240474435665,
 '수출': 188.65276039971215,
 '금지': -7.032411771763543,
 '제한': -5.562209690965512,
 '호국': -0.5005227705123151,
 '통제': 3.4900673602660146,
 '공급': 97.95416759507883,
 '우려': -7.276022830502384,
 '애플': 10.390740962926147,
 '수혜': -19.680355826748773,
 '가능성': -1.5852115935073612,
 '전기

- 뉴스의 평균 감성 점수 산출 뒤 컬럼에 추가

In [57]:
total = []
for i,w  in enumerate(train_data['nouns']):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab):
          continue
        else:
          sent_score = sent_score + vocab[j]
    total.append(sent_score/len(w))
train_data['sent_score'] = total

- sent_score가 감성사전의 평균 점수보다 높으면 1, 낮으면 0으로 새롭게 라벨링

In [58]:
sum = 0
for i in range(len(vocab)):
  sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in range(len(train_data)):
  if(train_data.iloc[i]['sent_score']>sent_mean):
    a_.append(1)
  else:
    a_.append(0)
train_data['sent_label'] = a_

In [59]:
train_data.head()

,일자,title,label,nouns,sent_score,sent_label
42839,2018-06-01,올해설비투자조원전망전년반도체디스플레이편,1,올해 설비 투자 전망 년 반도체 디스플레이 편,84.982611,1
21554,2020-07-30,반도체훈풍에로체시스템즈시설투자빛볼까,1,반도체 훈풍 로체 시스 템즈 시설 투자 빛,77.495790,1
32292,2019-06-14,분쟁피해서제조공장인도로덩달아반도체수출배,0,분쟁 피해 제조 공장 인도 덩달아 반도체 수출 배,101.812210,1
18520,2020-12-18,데이터확줄인이미지센서반도체올림픽서주목받다,1,데이터 확 이미지 센서 반도체 올림픽 주목,89.737679,1
21559,2020-07-29,수요와반도체사이클꼼꼼한분석으로대박리포트,0,수요 반도체 사이클 분석 대박 리포트,100.810614,1


### 7-2. test_data
- train_data와 동일한 방식으로 진행

In [60]:
up = 29202
down = 29141

up_ratio = up/(up+down)
down_ratio = down/(up+down)

for i,w in enumerate(test_data['nouns']):
    w = w.split(' ')
    if (test_data.iloc[i]['label']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] - up_ratio

In [61]:
# 감성 점수 확인
vocab

{'전함': -0.5005227705123151,
 '포템킨': -0.5005227705123151,
 '오데사': -0.5005227705123151,
 '네온사인': -0.5005227705123151,
 '반도체': 822.6097218167658,
 '위기': 4.680064446463012,
 '더독': -0.00209108204926034,
 '한국': 46.73936890458225,
 '기업': 45.37447508698668,
 '전자': 109.86563940833388,
 '부품': 34.09903501705419,
 '조달': 19.95817835901477,
 '삼성': -24.698781344806022,
 '진짜': -7.019865279467976,
 '약점': -3.0041821640985202,
 '바로': -0.00522770512315085,
 '비즈': -5.549663198669935,
 '일본': 33.69156539773396,
 '수도': 6.9362219974975385,
 '수도권': -7.027184066640389,
 '대학': 10.97386147438424,
 '학과': -9.547572116620689,
 '정원': 0.9958178359014793,
 '당선인': -1.5015683115369454,
 '침공': 2.989544589753699,
 '러시아': 3.9686337692610976,
 '반도체소자': -9.009409869221674,
 '품목': 3.3646024373103653,
 '대외': -8.029275148689655,
 '수출': 265.3795828120134,
 '금지': -11.543389952522165,
 '제한': -8.087825446068964,
 '호국': -1.0010455410246302,
 '통제': 3.4858851961674944,
 '공급': 102.93610201737073,
 '우려': -14.387895720137884,
 '애플': 11.349

- 뉴스의 평균 감성 점수 산출 뒤 컬럼에 추가

In [62]:
total = []
for i,w  in enumerate(test_data['nouns']):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab):
          continue
        else:
          sent_score = sent_score + vocab[j]
    total.append(sent_score/len(w))
test_data['sent_score'] = total

- sent_score가 감성사전의 평균 점수보다 높으면 1, 낮으면 0으로 새롭게 라벨링

In [63]:
sum = 0
for i in range(len(vocab)):
  sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in range(len(test_data)):
  if(test_data.iloc[i]['sent_score']>sent_mean):
    a_.append(1)
  else:
    a_.append(0)
test_data['sent_label'] = a_

In [64]:
test_data.head()

,일자,title,label,nouns,sent_score,sent_label
33993,2019-04-25,이재용비메모리육성공언반도체비전으로구체화,0,이재용 비메 모리 육성 공언 반도체 구체화,110.064615,1
21333,2020-08-04,삼성전자반도체사업장코로나검사소자체운영,0,삼성 전자 반도체 사업 코로나 검사 소 자체 운영,100.597949,1
1784,2022-01-21,삼성전자년만에인텔제치고세계반도체매출위달성,0,삼성 전자 인텔 제 세계 반도체 매출 위 달성,101.118420,1
2089,2022-01-11,쇼티지지속반도체여전히매력,1,쇼티 지속 반도체 매력,212.050490,1
36761,2019-01-30,거래소이젠종목추천까지금융반도체,1,거래소 젠 종목 추천 금융 반도체,141.587631,1


## 8. 워드 클라우드

### 8-1. train_data

In [65]:
# train_data > nouns 컬럼 > 0번째 데이터의 단어 출력
train_data.nouns.to_list()[0]

'올해 설비 투자 전망 년 반도체 디스플레이 편'

In [66]:
# train_data의 길이 확인
len(train_data.nouns.to_list())

43757

- train_data의 'nouns' 데이터 합치기

In [67]:
# 'nouns' 컬럼 데이터 리스트로 변환
nouns_list_train = []

for i in range(0, len(train_data.nouns.to_list())):
    noun = train_data.nouns.to_list()[i]
    nouns_list_train.append(noun)

In [68]:
nouns_list_train[:5]

['올해 설비 투자 전망 년 반도체 디스플레이 편',
 '반도체 훈풍 로체 시스 템즈 시설 투자 빛',
 '분쟁 피해 제조 공장 인도 덩달아 반도체 수출 배',
 '데이터 확 이미지 센서 반도체 올림픽 주목',
 '수요 반도체 사이클 분석 대박 리포트']

In [69]:
nouns_list_train[0]

'올해 설비 투자 전망 년 반도체 디스플레이 편'

In [70]:
nouns_list_train[0].split() + nouns_list_train[1].split()

['올해',
 '설비',
 '투자',
 '전망',
 '년',
 '반도체',
 '디스플레이',
 '편',
 '반도체',
 '훈풍',
 '로체',
 '시스',
 '템즈',
 '시설',
 '투자',
 '빛']

In [71]:
# WordCloud에 사용할 리스트 생성
nouns_total_train = []

for i in range(0, len(nouns_list_train)):
    nouns_total_train += nouns_list_train[i].split()

In [72]:
nouns_total_train

['올해',
 '설비',
 '투자',
 '전망',
 '년',
 '반도체',
 '디스플레이',
 '편',
 '반도체',
 '훈풍',
 '로체',
 '시스',
 '템즈',
 '시설',
 '투자',
 '빛',
 '분쟁',
 '피해',
 '제조',
 '공장',
 '인도',
 '덩달아',
 '반도체',
 '수출',
 '배',
 '데이터',
 '확',
 '이미지',
 '센서',
 '반도체',
 '올림픽',
 '주목',
 '수요',
 '반도체',
 '사이클',
 '분석',
 '대박',
 '리포트',
 '견제',
 '가격',
 '하락',
 '버팀목',
 '반도체',
 '험로',
 '예고',
 '한국',
 '주력',
 '산업',
 '코로나',
 '반도체',
 '위기',
 '팩트',
 '체크',
 '지금',
 '반도체',
 '업계',
 '상황',
 '반도체',
 '일반',
 '기계',
 '수출',
 '연속',
 '애플',
 '참여',
 '도시바',
 '반도체',
 '미국',
 '모닝',
 '터치',
 '서울',
 '반도체',
 '실적',
 '개선',
 '목표',
 '가도',
 '상향',
 '투자',
 '증권',
 '오늘',
 '키',
 '워드',
 '바이든',
 '반도체',
 '배터리',
 '등',
 '검토',
 '지시',
 '한국',
 '영향',
 '미치나',
 '조정은',
 '매수',
 '기회',
 '반도체',
 '자동차',
 '주목',
 '반도체',
 '장비',
 '업체',
 '매출',
 '급감',
 '전망',
 '무역',
 '전쟁',
 '수출',
 '규제',
 '변수',
 '아이',
 '케이',
 '규모',
 '반도체',
 '검사',
 '장비',
 '공급',
 '계약',
 '영하',
 '도',
 '혹',
 '부터',
 '영상',
 '도',
 '고온',
 '반도체',
 '삼성',
 '양산',
 '기업',
 '심리',
 '개월',
 '기준',
 '선',
 '밑',
 '반도체',
 '수출',
 '전망',
 '개월',
 '최저',
 '코로나',
 '우려',
 '수출',
 '개월

In [73]:
# 가장 많이 나온 단어 저장
counts = Counter(nouns_total_train)
tags = counts.most_common(30000)
tags

[('반도체', 45067),
 ('삼성', 9591),
 ('전자', 6615),
 ('수출', 4490),
 ('장비', 3571),
 ('투자', 3357),
 ('공급', 2957),
 ('하이닉스', 2735),
 ('계약', 2280),
 ('규모', 2235),
 ('실적', 1827),
 ('공장', 1810),
 ('주', 1753),
 ('영업', 1733),
 ('서울', 1670),
 ('산업', 1617),
 ('등', 1578),
 ('기업', 1572),
 ('한미', 1508),
 ('생산', 1421),
 ('최대', 1409),
 ('기술', 1338),
 ('위', 1323),
 ('시스템', 1306),
 ('용', 1304),
 ('제조', 1278),
 ('전망', 1249),
 ('시장', 1237),
 ('사업', 1233),
 ('매출', 1198),
 ('익', 1175),
 ('종합', 1159),
 ('소재', 1125),
 ('증가', 1123),
 ('내년', 1109),
 ('세계', 1089),
 ('한국', 1084),
 ('올해', 1032),
 ('상승', 1019),
 ('감소', 978),
 ('개월', 977),
 ('중국', 962),
 ('부족', 954),
 ('부진', 953),
 ('디스플레이', 941),
 ('성장', 940),
 ('호황', 919),
 ('글로벌', 910),
 ('업계', 900),
 ('하락', 896),
 ('메모리', 880),
 ('이익', 866),
 ('이재용', 851),
 ('정부', 828),
 ('특징', 806),
 ('업체', 802),
 ('자동차', 767),
 ('개발', 756),
 ('차', 749),
 ('지원', 739),
 ('검사', 735),
 ('코스닥', 732),
 ('인텔', 727),
 ('현대차', 723),
 ('뉴스', 705),
 ('협력', 687),
 ('회복', 673),
 ('것', 660),
 (

- 한글 폰트 사용

In [74]:
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

- 필요없는 단어 제거 : stopwords 설정

In [75]:
# 불용어 지정
stopwords = "램 사 액 첫 제 세 달 내년 반도체 분기 억 월 차 년 중 주 일 등 조 억원 용 익 위 개월 이재용 것 조원 초"
stop_list = stopwords.split()

# stop_list에 없는 단어만 추출 > 리스트를 만들 것
words_list = [word for word in nouns_total_train if word not in stop_list]

- 단어 빈도수 살펴보기

In [76]:
words = nltk.Text(words_list)
plt.figure(figsize=(20,12))
# 많이 사용된 단어 보여주기
words.plot(100)        
plt.show()

findfont: Font family ['Malgun Gothic'] not found. Falling back to DejaVu Sans.


In [77]:
# 빈도수가 200 이상인 단어 10개
data = words.vocab().most_common(200)
data[:10]

[('삼성', 9591),
 ('전자', 6615),
 ('수출', 4490),
 ('장비', 3571),
 ('투자', 3357),
 ('공급', 2957),
 ('하이닉스', 2735),
 ('계약', 2280),
 ('규모', 2235),
 ('실적', 1827)]

- train_data 워드클라우드 생성

In [78]:
wc = WordCloud(
    font_path='c:/Windows/Fonts/malgun.ttf',
    relative_scaling=0.2, background_color='white',
    colormap='coolwarm'
).generate_from_frequencies(dict(data))


OSError: ignored

In [79]:
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis('off')
plt.show()

NameError: ignored

### 8-2. test_data

In [80]:
# test_data > nouns 컬럼 > 0번째 데이터의 단어
test_data.nouns.to_list()[0]


# 'nouns' 컬럼 데이터 리스트로 변환
nouns_list_test = []

for i in range(0, len(test_data.nouns.to_list())):
    noun = test_data.nouns.to_list()[i]
    nouns_list_test.append(noun)


# 'nouns' 컬럼 데이터 단어 데이터 합치기
nouns_total_test = []

for i in range(0, len(nouns_list_test)):
    nouns_total_test += nouns_list_test[i].split()

# 가장 많이 나온 단어 저장
counts = Counter(nouns_total_test)
tags = counts.most_common(30000)
tags


# 불용어 지정
stopwords = "올해 작년 램 사 액 첫 제 세 달 내년 반도체 분기 억 월 차 년 중 주 일 등 조 억원 용 익 위 개월 이재용 것 조원 초"
stop_list = stopwords.split()

# stop_list에 없는 단어만 추출 > 리스트를 만들 것
words_list = [word for word in nouns_total_test if word not in stop_list]

words = nltk.Text(words_list)
plt.figure(figsize=(20,12))
# 많이 사용된 단어 보여주기
words.plot(100)        
plt.show()

In [81]:
# 빈도수가 200 이상인 단어 10개
data = words.vocab().most_common(200)
data[:10]

[('삼성', 3223),
 ('전자', 2250),
 ('수출', 1479),
 ('장비', 1204),
 ('투자', 1090),
 ('공급', 991),
 ('하이닉스', 894),
 ('계약', 757),
 ('규모', 697),
 ('실적', 627)]

- test_data 워드클라우드 생성

In [82]:
wc = WordCloud(
    font_path='c:/Windows/Fonts/malgun.ttf',
    relative_scaling=0.2, background_color='white',
    colormap='coolwarm'
).generate_from_frequencies(dict(data))

plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis('off')
plt.show()

OSError: ignored

## 9. 모델링

### 9-1. train_set, test_set을 나눠준 다음, 형태소를 분류하고 X와 Y값을 지정

In [124]:
train_set, test_set = train_test_split(train_data, test_size=0.2)
train_set.dropna(how='any', inplace=True)
test_set.dropna(how='any', inplace=True)
mecab = Mecab()
train_set['tokenized'] = train_set['title'].apply(mecab.morphs)
test_set['tokenized'] = test_set['title'].apply(mecab.morphs)
X_train = train_set['tokenized'].values
Y_train = train_set['sent_label'].values
X_val = test_set['tokenized'].values
Y_val = test_set['sent_label'].values

### 9-2. keras의 Tokenizer를 사용해 텍스트를 시퀀스 벡터로 변환
    - 모델의 input으로는 텍스트가 들어갈 수 없다

In [87]:
# vocab_size는 단어 집합의 크기
vocab_size = 5
tokenizer = Tokenizer(vocab_size, oov_token='OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

### 9-3. 일정한 Input 형태를 만들기 위해 적절한 길이를 지정하고 padding과정으로 마무리

In [90]:
def below_threshold_len(max_len, nested_list):
    count = 0
    for s in nested_list:
        if (len(s) <= max_len):
            count = count + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count/len(nested_list))*100))

max_len = 30
below_threshold_len(max_len, X_train)
X_train = pad_sequences(X_train, maxlen=max_len)
X_val = pad_sequences(X_val, maxlen=max_len)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 100.0


### 9-4. 모델 구현

In [96]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [167]:
from tensorflow.keras.models import load_model

In [100]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=15, callbacks=[es,mc], batch_size=20, validation_split=0.2)

Epoch 1/15
1401/1401 [==============================] - ETA: 0s - loss: 0.0110 - acc: 0.9986
Epoch 1: val_acc improved from -inf to 0.99886, saving model to best_model.h5
1401/1401 [==============================] - 68s 44ms/step - loss: 0.0110 - acc: 0.9986 - val_loss: 0.0099 - val_acc: 0.9989
Epoch 2/15
1401/1401 [==============================] - ETA: 0s - loss: 0.0078 - acc: 0.9991
Epoch 2: val_acc did not improve from 0.99886
1401/1401 [==============================] - 59s 42ms/step - loss: 0.0078 - acc: 0.9991 - val_loss: 0.0093 - val_acc: 0.9989
Epoch 3/15
1401/1401 [==============================] - ETA: 0s - loss: 0.0077 - acc: 0.9991
Epoch 3: val_acc did not improve from 0.99886
1401/1401 [==============================] - 59s 42ms/step - loss: 0.0077 - acc: 0.9991 - val_loss: 0.0092 - val_acc: 0.9989
Epoch 4/15
1401/1401 [==============================] - ETA: 0s - loss: 0.0076 - acc: 0.9991
Epoch 4: val_acc did not improve from 0.99886
1401/1401 [==========================

In [168]:
loaded_model = load_model('best_model.h5')

In [181]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = mecab.morphs(new_sentence) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정입니다.".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정입니다.".format((1 - score) * 100))

In [180]:
sentiment_predict('코스피 안정세에 반도체·2차전지 등 성장주 주가 상승률↑')

99.98% 확률로 긍정입니다.


In [184]:
sentiment_predict('정상화 기대했는데…日 지진에 자동차·반도체 공장 또 "셧다운"')

99.98% 확률로 긍정입니다.
